<a href="https://pytorch.org/"><img src="https://raw.githubusercontent.com/pytorch/pytorch/master/docs/source/_static/img/pytorch-logo-dark.png" width="600px"/></a>

# **PyTorch** (2/2): Audio Classification

In this tutorial, we will train a neural network for audio classification using transfer learning. 

Some of the code in this example is based on [this](https://github.com/hasithsura/Environmental-Sound-Classification).

In [1]:
%matplotlib widget

In [2]:
from helpers.all import *

In [3]:
# select device to run on
use_cuda = torch.cuda.is_available()
device = torch.device('cuda' if use_cuda else 'cpu')
print(device)

cpu


# Preparing the Dataset

We will use the [ESC-50](https://github.com/karolpiczak/ESC-50) (Dataset for Environmental Sound Classification) in our experiment. If you cloned this repository including the `--recurse-submodules` flag, it is already downloaded and should be present at

In [4]:
print(DATASET_ROOT)

A:\code\git\jupyter-pytorch-tutorial\train\dataset\ESC-50


The dataset includes a `.csv` file that contains information about the samples in the dataset (e.g. which category it belongs to, etc.). We use `pandas` to read the `.csv` into a `DataFrame`.

In [5]:
df = pd.read_csv(DATASET_ROOT/'meta/esc50.csv')

Let's take a look at the first few lines...

In [6]:
df.head()

,filename,fold,target,category,esc10,src_file,take
0,1-100032-A-0.wav,1,0,dog,True,100032,A
1,1-100038-A-14.wav,1,14,chirping_birds,False,100038,A
2,1-100210-A-36.wav,1,36,vacuum_cleaner,False,100210,A
3,1-100210-B-36.wav,1,36,vacuum_cleaner,False,100210,B
4,1-101296-A-19.wav,1,19,thunderstorm,False,101296,A


Fortunately, in the docs of the dataset it states

> The dataset has been prearranged into 5 folds for comparable cross-validation, making sure that fragments from the same original source file are contained in a single fold.

This means, that we can just pick one of the folds to use as our validation set. We create a new column `is_valid` that is true for fold 5.

In [7]:
df['is_valid'] = df['fold'] == 5

In [8]:
df

,filename,fold,target,category,esc10,src_file,take,is_valid
0,1-100032-A-0.wav,1,0,dog,True,100032,A,False
1,1-100038-A-14.wav,1,14,chirping_birds,False,100038,A,False
2,1-100210-A-36.wav,1,36,vacuum_cleaner,False,100210,A,False
3,1-100210-B-36.wav,1,36,vacuum_cleaner,False,100210,B,False
4,1-101296-A-19.wav,1,19,thunderstorm,False,101296,A,False
...,...,...,...,...,...,...,...,...
1995,5-263831-B-6.wav,5,6,hen,False,263831,B,True
1996,5-263902-A-36.wav,5,36,vacuum_cleaner,False,263902,A,True
1997,5-51149-A-25.wav,5,25,footsteps,False,51149,A,True
1998,5-61635-A-8.wav,5,8,sheep,False,61635,A,True


# Training Phase

Instead of training a model entirely from scratch, we will use the concept of [_transfer learning_](https://en.wikipedia.org/wiki/Transfer_learning). This means we are starting with a pretrained model from a different but related problem and _fine tuning_ it to our specific problem.

Many of the break-through achievements in deep learning are coming from the domain of computer vision (e.g. image classification). By transforming audio into images (spectrograms) we can use those very same networks to perform audio classification.

## Building the Input Transformation Pipeline

A fast.ai [`DataBlock`](https://docs.fast.ai/data.block.html#DataBlock) is a convenient way of organizing our data into a form that can be used during the training phase. 

> By itself, a DataBlock is just a blue print on how to assemble your data. It does not do anything until you pass it a source. You can choose to then convert that source into a Datasets or a DataLoaders by using the DataBlock.datasets or DataBlock.dataloaders method.

If you want to know more about DataBlocks, have a look at this [tutorial](https://docs.fast.ai/tutorial.datablock.html).

In [9]:
params = load_params()
audio2spec = AudioToSpec.from_cfg(AudioConfig.BasicMelSpectrogram(**params))
normalize = AudioNormalize()

In [10]:
block = DataBlock(
    blocks=(AudioBlock, CategoryBlock),
    splitter = ColSplitter(),
    get_x = ColReader('filename', pref=DATASET_ROOT/'audio'),
    get_y = ColReader('category'),
    item_tfms=[normalize],
    batch_tfms=[audio2spec],
)

In [11]:
dls = block.dataloaders(df, num_workers=0) # on windows, num_workers needs to be set to 0
dls.show_batch(figsize=(7,7))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Create/Prepare Model

Now, we create a `learner` object by passing it our DataLoaders object and using a pretrained ResNet18 as our architechture.

If you run this the first time, it will download the pretrained ResNet18 parameters.

In [12]:
learn = cnn_learner(dls, models.resnet18, metrics=[accuracy], pretrained=True)

The ResNet models work on RGB images, i.e. the input layer expects an image with three channels. 

In [13]:
learn.model[0][0].in_channels

3

Our spectrograms only have a single channel, so we need to modify the first layer of the model slightly to take our spectrograms as input.

In [14]:
make_xresnet_grayscale(learn.model)

If CUDA is available, we now transfer our model to the GPU. (If you do not have CUDA available, this cell will do nothing).

In [15]:
learn.model.to(device);

## Training the Model

... and finally we call `fine_tune` to train the model for 5 epochs.

In [16]:
learn.fine_tune(1, 3e-3)

KeyboardInterrupt: 

## Interpreting the Training Results

After 5 epochs of training, you should see an accuracy of ~65 %. This may not seem very good but keep in mind that we just trained the model for a couple of minutes. Also, we are working with 50 output classes! 

If you have a look at the [original experiment](https://github.com/hasithsura/Environmental-Sound-Classification), you can see that with some further training it is easy to achieve an accuracy of over 85 %.

To get a better understanding of how well our model is doing, we can create a `ClassificationInterpretation` object from our learner and plot the confusion matrix.

In [72]:
interp = ClassificationInterpretation.from_learner(learn)
interp.plot_confusion_matrix(figsize=(10, 10))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Or we can take a look at which pairs of categories the model has the most trouble with:

In [73]:
interp.most_confused(min_val=2)

[('engine', 'washing_machine', 6),
 ('airplane', 'wind', 4),
 ('can_opening', 'sneezing', 4),
 ('fireworks', 'footsteps', 4),
 ('clock_tick', 'hand_saw', 3),
 ('crying_baby', 'cat', 3),
 ('helicopter', 'washing_machine', 3),
 ('laughing', 'pig', 3),
 ('mouse_click', 'crackling_fire', 3),
 ('pouring_water', 'brushing_teeth', 3),
 ('water_drops', 'mouse_click', 3),
 ('breathing', 'mouse_click', 2),
 ('breathing', 'snoring', 2),
 ('can_opening', 'crackling_fire', 2),
 ('chainsaw', 'vacuum_cleaner', 2),
 ('chirping_birds', 'crow', 2),
 ('clock_tick', 'footsteps', 2),
 ('coughing', 'sneezing', 2),
 ('crickets', 'clock_alarm', 2),
 ('dog', 'laughing', 2),
 ('dog', 'sneezing', 2),
 ('door_wood_creaks', 'cat', 2),
 ('drinking_sipping', 'pig', 2),
 ('fireworks', 'crackling_fire', 2),
 ('frog', 'crow', 2),
 ('helicopter', 'airplane', 2),
 ('hen', 'fireworks', 2),
 ('laughing', 'frog', 2),
 ('pig', 'glass_breaking', 2),
 ('pouring_water', 'toilet_flush', 2),
 ('rooster', 'cat', 2),
 ('thunderstor

# Saving the Trained Model

For running inference in our C++ application, we need to do two things:

## Export the Model as TorchScript

[TorchScript](https://pytorch.org/tutorials/beginner/Intro_to_TorchScript_tutorial.html#) is a different representation of a PyTorch model that can be loaded and run in C++. The process of transforming our trained PyTorch model into a TorchScript module is called _tracing_. 

Tracing a model will invoke it with some _example input_ and recorde all the operations that occur during its execution. These recorded operations will then be saved into a static representation of our model (this is usually called _graph_).

In our case, all we need to do is call `torch.jit.trace()`, passing our model and some dummy input as arguments. As dummy input, we will use a random tensor that has the same shape as our spectrograms during training (`[1, 128, 157]`, i.e. `[n_channels, n_mels, n_frames]`), 

In [22]:
# move model back to CPU memory if we were training on GPU
if use_cuda:
    learn.model.to('cpu')
    
# create some dummy input 
dummy_input = torch.randn([1, 1, 128, 157])
    
# process the trace
traced_script_module = torch.jit.trace(learn.model, dummy_input)

In [74]:
# create output dir if it doesn't exist
if not MODEL_DIR.exists():
    MODEL_DIR.mkdir()

In [75]:
torch.jit.save(traced_script_module, str(MODEL_DIR/'esc-model.pt'))

## Save the Vocabulary

Our model does not directly output a string with the predicted category. Rather, it outputs a vector containing an unnormalized score for each of the categories in our vocabulary. 

In [84]:
learn.model(torch.randn(1, 1, 128, 157).to(device))

tensor([[ 1.4494,  0.3502, -0.4341, -1.6450, -0.2474, -1.0542, -1.7261, -0.8983,
         -0.8715,  2.5045,  1.3310,  0.5275, -0.7423, -2.0549,  1.4351,  0.3269,
         -0.0683, -1.9899, -1.4474, -1.3243, -1.0975, -0.9657,  1.9727, -0.8710,
          2.5978, -0.0487,  1.2318,  0.1464,  0.8132, -1.2667, -1.0077,  2.8095,
         -0.9545, -0.0314,  1.1057,  0.2084,  2.0173, -0.8538,  1.4979, -0.7353,
          1.4881, -0.8945, -0.9431,  0.6804,  1.4942,  2.7140,  1.3923,  1.0871,
          0.3736,  1.1016]], device='cuda:0', grad_fn=<MmBackward>)

These are oredered in the same way as our vocabulary, so converting the model output to the string of the predicted category is straightforward: we just need to find the index of the item with the highest score (using `argmax()`) and use this as an index into our vocabulary vector.

To be able to perform this mapping directly in out C++ application, we our vocabulary into a header file that will automatically be placed in the C++ source directory:

In [79]:
dls.vocab

['airplane', 'breathing', 'brushing_teeth', 'can_opening', 'car_horn', 'cat', 'chainsaw', 'chirping_birds', 'church_bells', 'clapping', 'clock_alarm', 'clock_tick', 'coughing', 'cow', 'crackling_fire', 'crickets', 'crow', 'crying_baby', 'dog', 'door_wood_creaks', 'door_wood_knock', 'drinking_sipping', 'engine', 'fireworks', 'footsteps', 'frog', 'glass_breaking', 'hand_saw', 'helicopter', 'hen', 'insects', 'keyboard_typing', 'laughing', 'mouse_click', 'pig', 'pouring_water', 'rain', 'rooster', 'sea_waves', 'sheep', 'siren', 'sneezing', 'snoring', 'thunderstorm', 'toilet_flush', 'train', 'vacuum_cleaner', 'washing_machine', 'water_drops', 'wind']

In [85]:
write_vocab_cpp_header(dls.vocab)

For good measure, we will also export the `vocab` list as a [pickle](https://docs.python.org/3/library/pickle.html) file, so we can easily reuse in some other Python script if we need to.

In [81]:
with open(TRAIN_DIR/'esc-model-vocab.pkl', 'wb') as f:
    pickle.dump(dls.vocab, f)

# Inference Phase

## Setting up a Test Dataset

The `DataLoader` class we used earlier automatically takes care of splitting the dataset into training and validation sets. For testing purposes, however, we cannot use samples from either of these. This leaves us with two options:

1. splitting off some items of the dataset for testing before creating the `DataLoader` for training or
2. using new samples, e.g. from a different dataset.

Since we are already done with training, we will go for the latter option. I have manually selected some sound examples from [freesound.org](https://freesound.org/):

* [an airplane](https://freesound.org/people/AurelioSons/sounds/207457/),
* [a guy sneezing](https://freesound.org/people/InspectorJ/sounds/368804/),
* [a toilet flush](https://freesound.org/people/InspectorJ/sounds/404329/),
* [a dog bark](https://freesound.org/people/Juan_Merie_Venter/sounds/327666/),
* [some sheeps](https://freesound.org/people/zachrau/sounds/362283/),
* [a child laughing](https://freesound.org/people/Teumova/sounds/439667/),
* [some mouse clicking](https://freesound.org/people/Masgame/sounds/347544/),
* [a siren](https://freesound.org/people/Kingrock2009/sounds/544376/),
* [a coffee machine](https://freesound.org/people/Acekat13/sounds/515685/),
* [a harp](https://freesound.org/people/pryght%20one/sounds/27130/) and
* [a synth sound](https://freesound.org/people/Erokia/sounds/550708/).

Excecpt for last three examples, all samples belong to categories known to our model.

To begin with, we need to manually label our test data, i.e. we create a dictionary that maps filenames to their category.

In [86]:
test_files_to_label = {
    '404329__inspectorj__toilet-flush-european-distant-lid-up.wav': 'toilet_flush',
    '327666__juan-merie-venter__dog-bark.wav': 'dog',
    '368804__inspectorj__sneeze-single-b.wav': 'sneezing',
    '347544__masgame__mouse-click-sounds.wav': 'mouse_click',
    '207457__aureliosons__avion-de-elices.wav': 'airplane',
    '27130__pryght-one__harp.wav': 'harp (unknown category)',
    '439667__teumova__child-laughing.wav': 'laughing',
    '515685__acekat13__adriana-lopez-coffee-machine.wav': 'coffee_machine (unknown category)',
    '544376__kingrock2009__siren-1.wav': 'siren',
    '362283__zachrau__sheep-bleating.wav': 'sheep',
    '550708__erokia__msfxp9-14-synth-loop-100-bpm.wav': 'synth_loop (unknown category)'
}

Our model expects spectrograms, not the raw waveforms. During training, this transformation was automatically handled by the `DataLoader` class. 

We don't need all the functionality offered by the `DataLoader`, we just want to transform all test samples. For this, we can use the `Pipeline` class. As the name suggests, this is just a pipeline of `Transform`s.

Because we are not dealing with a homogenous dataset as during training, we need to add a couple of transforms to make sure that our test samples have the same properties as those from the training dataset:
* `Resample`: resample to 16 kHz
* `DownmixMono`: downmix stereo signals to mono
* `ResizeSignal`: resize to exactly 5 s in length (padding or clipping if needed)

Let's define our transform pipeline, using the `normalize` and `audio2spec` transforms we defined earlier:

In [88]:
resize_to_5s = ResizeSignal(5000, pad_mode=AudioPadType.Zeros_After)
resample_to_16khz = Resample(params['sample_rate'])
downmix = DownmixMono()

transforms = Pipeline([AudioTensor.create, resize_to_5s, downmix, resample_to_16khz, normalize, audio2spec])

We build a new dictionary mapping our the label of each test sample to the output of the `transforms` pipeline (i.e. our spectrograms):

In [89]:
label_to_spectrogram = {label: transforms(TEST_DIR/'data'/filename) for filename, label in test_files_to_label.items()}

We save the spectrograms into a TorchScript module to be able to load it from the C++ application.

To have access to the labels from within the C++ application, we register each of our spectrograms as a _named buffer_, using the corresponding label as name.

In [30]:
# save test samples as named buffers in a module
container = torch.nn.Module()
for l, s in label_to_spectrogram.items():
    container.register_buffer(l, torch.tensor(s))
    
# save to torch script module
torch.jit.save(torch.jit.script(container), str(TEST_DIR/'inputs.pt'))

## Building the C++ application

For demo purposes, we will be running the inference part in C++.

In IPython (which is running in the backend kernel of this notebook), we can run any command-line command by prefixing it with a `!`. 

This way we can just build and call the C++ application from right here within the notebook! We can also reuse any of our currently defined variables by including them in `{`curly braces`}`.

### Checking for `cmake`

Make sure you have `cmake` installed and its executable can be found. If everything is set up correctly, you should see the version output from `cmake` when executing the following cell.

In [ ]:
!cmake --version

### Generating Build System

After making sure that our build directory exists, we call `cmake` to generate our build system.

In [31]:
# create build directory if it does not exist
if not CPP_BUILD_DIR.exists():
    CPP_BUILD_DIR.mkdir()

In [32]:
!cmake -S {CPP_SOURCE_DIR} -B {CPP_BUILD_DIR} -DCMAKE_PREFIX_PATH={torch.utils.cmake_prefix_path}

-- Selecting Windows SDK version 10.0.18362.0 to target Windows 10.0.19042.
-- Caffe2: CUDA detected: 11.0
-- Caffe2: CUDA nvcc is: C:/Program Files/NVIDIA GPU Computing Toolkit/CUDA/v11.0/bin/nvcc.exe
-- Caffe2: CUDA toolkit directory: C:/Program Files/NVIDIA GPU Computing Toolkit/CUDA/v11.0
-- Caffe2: Header version is: 11.0
-- Found cuDNN: v8.1.0  (include: C:/Program Files/NVIDIA GPU Computing Toolkit/CUDA/v11.0/include, library: C:/Program Files/NVIDIA GPU Computing Toolkit/CUDA/v11.0/lib/x64/cudnn.lib)
-- Autodetected CUDA architecture(s):  6.1
-- Added CUDA NVCC flags for: -gencode;arch=compute_61,code=sm_61
-- Configuring done
-- Generating done
-- Build files have been written to: A:/source/git/jupyter-pytorch-tutorial/inference-cpp/build


### Building the Application

Next, we can trigger the actual build.

**WARNING**: note that you cannot just switch the build config to `Debug`, as the `libtorch` libraries for `Debug`/`Release` are not ABI-compatible. 


In [33]:
!cmake --build {CPP_BUILD_DIR} --config Release

Microsoft (R) Build Engine version 16.8.3+39993bd9d for .NET Framework
Copyright (C) Microsoft Corporation. All rights reserved.

  Checking Build System
  Building Custom Rule A:/source/git/jupyter-pytorch-tutorial/inference-cpp/CMakeLists.txt
  main.cpp
     Creating library A:/source/git/jupyter-pytorch-tutorial/inference-cpp/build/Release/esc-app.lib and object A:/source/git/jupyter-pytorch-tutorial/inference-cpp/build/Release/esc-app.exp
  esc-app.vcxproj -> A:\source\git\jupyter-pytorch-tutorial\inference-cpp\build\Release\esc-app.exe
  Building Custom Rule A:/source/git/jupyter-pytorch-tutorial/inference-cpp/CMakeLists.txt


## Running Inference

After building the application successfully, we can call it to run inference using our trained model

In [34]:
esc_executable = CPP_BUILD_DIR/'esc-app'

if platform.system() == 'Windows':
    esc_executable = ((esc_executable.parent/'Release')/esc_executable.name).with_suffix('.exe')

In [ ]:
!{esc_executable}

In [36]:
model_path = MODEL_DIR/'esc-model.pt'
inputs_path = TEST_DIR/'inputs.pt'
outputs_path = TEST_DIR/'outputs.pt'

In [ ]:
!{esc_executable} {model_path} {inputs_path} {outputs_path}

# Using `ipywidgets` to interactively explore our results

In [91]:
outputs = next(torch.load(outputs_path).parameters())

In [92]:
outputs[0]

tensor([ 0.4219, -1.5189, -2.4032, -2.7568, -0.7769, -1.1153,  1.4103, -0.4580,
         1.2483,  0.6536,  2.5087,  0.5938, -1.4581, -1.4538,  0.9915,  0.9075,
        -1.5791, -0.3977, -3.3914, -2.3287, -2.9719, -1.9985,  2.4453, -3.3235,
        -1.8727, -2.0996, -0.3472,  2.7163, -0.1663, -0.5855,  2.8562, -0.3234,
         0.1062, -1.1269,  2.6910,  0.2615,  2.2524, -0.8639,  4.1116,  1.1256,
        -0.6318, -0.7213,  0.4126, -0.1176,  7.1047,  2.3504,  3.2012,  0.3888,
        -1.5762, -0.6573], grad_fn=<SelectBackward>)

As the output contains unnormalized scores we use `softmax` to convert them into probabilities.

In [94]:
outputs_prob = torch.softmax(outputs.detach(), dim=1)
outputs_prob[0]

tensor([1.0657e-03, 1.5303e-04, 6.3201e-05, 4.4377e-05, 3.2139e-04, 2.2912e-04,
        2.8636e-03, 4.4207e-04, 2.4353e-03, 1.3435e-03, 8.5886e-03, 1.2656e-03,
        1.6261e-04, 1.6332e-04, 1.8837e-03, 1.7319e-03, 1.4409e-04, 4.6954e-04,
        2.3526e-05, 6.8085e-05, 3.5786e-05, 9.4724e-05, 8.0614e-03, 2.5178e-05,
        1.0742e-04, 8.5621e-05, 4.9389e-04, 1.0570e-02, 5.9182e-04, 3.8916e-04,
        1.2157e-02, 5.0576e-04, 7.7721e-04, 2.2646e-04, 1.0306e-02, 9.0775e-04,
        6.6472e-03, 2.9461e-04, 4.2665e-02, 2.1541e-03, 3.7155e-04, 3.3976e-04,
        1.0559e-03, 6.2134e-04, 8.5102e-01, 7.3315e-03, 1.7166e-02, 1.0311e-03,
        1.4450e-04, 3.6221e-04])

In [95]:
outputs_prob[0].sum()

tensor(1.0000)

Using `ipywidgets`, we can create GUI widgets to interactively explore our results. At first, we define a dropdown menu that contains the labels from our test samples:

In [96]:
test_labels = list(test_files_to_label.values())

In [97]:
dropdown = widgets.Dropdown(options = test_labels, description='Test sample: ')
dropdown

Dropdown(description='Test sample: ', options=('toilet_flush', 'dog', 'sneezing', 'mouse_click', 'airplane', '…

Depending on our selection in this dropdown, we want print the output probabilities from our model. We can do this by using the `Output` widget. It acts as a context manager, capturing all output that is produced during the context.

In [98]:
output = widgets.Output()
output

Output()

Right now, however, this is just an empty placeholder. 

We need to create an event handler that calls the `print_top_results` using the active index from the dropdown menu and register it as an observer of the `index` trait on the dropdown menu object.

In [99]:
def on_dropdown(_):
    output.clear_output()
    with output:
        print_top_results(dropdown.index, outputs_prob, test_labels, dls.vocab)
    
dropdown.observe(on_dropdown, names=['index'])

**Select some items from the Dropdown menu!**

We can also combine these widgets into Layouts:

In [100]:
output_plot = widgets.Output()
def on_dropdown(_):
    output_plot.clear_output()
    with output_plot:
        fig, _ = plot_top_results(outputs_prob[dropdown.index], dls.vocab, show_max=10, figsize=(5,3))
        fig.canvas.toolbar_visible = False
        fig.tight_layout()
dropdown.observe(on_dropdown, names=['index'])

widgets.VBox([dropdown, output_plot])

# End-to-End Example

Using our trained model and `ipywidgets`, we can even build a small application right here from inside the notebook (inspired an example from the [fast.ai course](https://course.fast.ai/videos/?lesson=3https://course.fast.ai/videos/?lesson=3)).

Select 

Note: selecting an audio file longer than 5 s and classifying it multiple times might yield different results due to a random cropping in the `ResizeSignal` transform.Note: selecting an audio file longer than 5 s and classifying it multiple times might yield different results due to a random cropping in the `ResizeSignal` transform.

In [101]:
model = torch.load(MODEL_DIR/'esc-model.pt')

In [102]:
btn_upload = widgets.FileUpload(accept='.wav')
btn_classify = widgets.Button(description='Classify', layout=widgets.Layout(width='300px'))
audio_player = widgets.Audio(autoplay=False, loop=False, layout=widgets.Layout(width='300px'))
label_upload = widgets.Label(value='No file selected')
label_class = widgets.Label(layout=widgets.Layout(width='300px'))
output = widgets.Output()

def on_click_upload(_):
    audio_player.value = btn_upload.data[-1]
    label_upload.value = btn_upload.metadata[-1]['name']
    btn_classify.description = 'Classify'
    
def classify(_):
    tmp = tempfile.NamedTemporaryFile(delete=False)
    tmp.file.write(btn_upload.data[-1]); tmp.close()

    # load tensor from tmp file and apply transforms
    spec = torch.tensor(transforms(tmp.name))
    probs = model(spec.unsqueeze(0)).softmax(dim=1).squeeze().detach()
    result_idx = probs.argmax()
    output.clear_output()
    with output:
        fig, _ = plot_top_results(probs, dls.vocab, show_max=10, figsize=(5, 3))
        fig.canvas.header_visible = False
        fig.canvas.toolbar_visible = False
        fig.tight_layout()

    tmp.close(); os.unlink(tmp.name)

def reset_button(_):
    btn_upload.value.clear()
    btn_upload._counter = 1

btn_upload.observe(on_click_upload, names=['data'])
btn_classify.on_click(classify)
btn_upload.observe(reset_button, names=['value'])

widgets.VBox([widgets.HBox([btn_upload, label_upload]), audio_player, btn_classify, output])

# Congratulations!

You have successfully 

* trained a neural network to classify audio examples into 50 categories
* built a C++ application to load and run the trained model
* created small interactive applications to analyze the results